In [4]:
# !pip install seaborn

     |████████████████████████████████| 285 kB 2.1 MB/s eta 0:00:01


In [1]:
!git clone https://github.com/ultralytics/yolov3.git

Cloning into 'yolov3'...
remote: Enumerating objects: 9783, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 9783 (delta 0), reused 2 (delta 0), pack-reused 9779
Receiving objects: 100% (9783/9783), 8.80 MiB | 7.43 MiB/s, done.
Resolving deltas: 100% (6656/6656), done.


In [2]:
import sys
import os
sys.path.append('./yolov3')

In [5]:
from pycocotools.coco import COCO
import numpy as np
import cv2

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from utils.loss import ComputeLoss

import yaml
import torch
from models.yolo import Model
from pycocotools.cocoeval import COCOeval
import pandas as pd

from torch.utils.data import DataLoader, Dataset
import torchvision
from utils.general import coco80_to_coco91_class, check_dataset, check_file, check_img_size, check_requirements, \
    box_iou, non_max_suppression, scale_coords, xyxy2xywh, xywh2xyxy, set_logging, increment_path, colorstr
from tqdm import tqdm

In [6]:
class CustomDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''

    def __init__(self, annotation, data_dir, transforms=None, image_size=512):
        super().__init__()
        self.data_dir = data_dir
        self.coco = COCO(annotation)
        self.image_size = image_size
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None
        }
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.coco.getImgIds(imgIds=index)
        image_info = self.coco.loadImgs(image_id)[0]
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)

        # boxes (x, y, w, h)
        boxes = np.array([x['bbox'] for x in anns])

        # boxes (x_min, y_min, x_max, y_max)
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        boxes[:, 0] /= int(self.image_size)
        boxes[:, 1] /= int(self.image_size)
        boxes[:, 2] /= int(self.image_size)
        boxes[:, 3] /= int(self.image_size)
        
        labels = list([x['category_id'] for x in anns])
        areas = np.array([x['area'] for x in anns])
        areas = torch.as_tensor(areas, dtype=torch.float32)
        
        is_crowds = np.array([x['iscrowd'] for x in anns])
        is_crowds = torch.as_tensor(is_crowds, dtype=torch.int64)
        
        segmentation = np.array([x['segmentation'] for x in anns], dtype=object)

        
        target = {'boxes': boxes, 'labels': labels, 'image_id': torch.tensor([index]), 'area': areas,
                  'iscrowd': is_crowds}

        # transform
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            target['boxes'] = [list(box) for box in sample['bboxes']]

        result = []
        labels_out = torch.zeros((len(labels), 6))
        for i in range(len(labels)):
            result.append(
                [labels[i], target['boxes'][i][0], target['boxes'][i][1], target['boxes'][i][2], target['boxes'][i][3]])
        result = np.array(result)

        if len(labels):
            labels_out[:, 1:] = torch.from_numpy(result)

        return image, labels_out, image_id
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())

In [7]:
def get_train_transform():
    return A.Compose([
        A.Resize(512, 512),
        A.Flip(p=0.5),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [8]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0


def collate_fn(batch):
    img, label, image_id = zip(*batch)  # transposed
    for i, l in enumerate(label):
        l[:, 0] = i  # add target image index for build_targets()
    return torch.stack(img, 0), torch.cat(label, 0), image_id


In [9]:
def train_fn(num_epochs, train_data_loader, optimizer, model, device):
    model.gr = 1.0
    itr = 1
    loss_hist = Averager()
    compute_loss = ComputeLoss(model)
    for epoch in range(num_epochs):
        loss_hist.reset()

        for images, targets, image_ids in tqdm(train_data_loader):
            images = torch.as_tensor([image.numpy() for image in images]).to(device)
            pred = model(images)

            losses, loss_items = compute_loss(pred, targets.to(device))  # loss scaled by batch_size

            loss_value = losses.item()

            loss_hist.send(loss_value)

            # backward
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()


        print(f"Epoch #{epoch} loss: {loss_hist.value}")
        torch.save(model.state_dict(), f'./yolov3/weights/epoch_{epoch+1}.pth')

In [19]:
def main():
    data_dir = '../../input/data'
    annotation = '../../input/data/train.json'
    train_dataset = CustomDataset(annotation, data_dir, get_train_transform())

    train_data_loader = DataLoader(
        train_dataset,
        batch_size=4,
        shuffle=False,
        num_workers=4,
        collate_fn=collate_fn
    )
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    # load a model;

    num_classes = 11
    model = Model(cfg='./yolov3/models/yolov3.yaml', ch=3, nc=num_classes)  # create

    # get number of input features for the classifier
    model.to(device)

    # Hyperparameters
    with open('./yolov3/data/hyp.scratch.yaml') as f:
        hyp = yaml.load(f, Loader=yaml.SafeLoader)  # load hyps
    model.hyp = hyp
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

    num_epochs = 12

    train_fn(num_epochs, train_data_loader, optimizer, model, device)


In [20]:
if __name__ == '__main__':
    main()

loading annotations into memory...
Done (t=4.72s)
creating index...
index created!


100%|██████████| 655/655 [05:56<00:00,  1.84it/s]


Epoch #0 loss: 1.1051222209740459


100%|██████████| 655/655 [05:56<00:00,  1.83it/s]


Epoch #1 loss: 1.0143869513681314


...

Epoch #9 loss: 1.0011018390319388


100%|██████████| 655/655 [06:02<00:00,  1.81it/s]


Epoch #10 loss: 0.9625316589018543


100%|██████████| 655/655 [06:02<00:00,  1.81it/s]


Epoch #11 loss: 0.9770777327186279


In [18]:
cd mmdetection_trash

/opt/ml/code/mmdetection_trash
